In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
print(dataset.isnull().values.any())

In [ ]:
encoded_columns = []
for i in ['sex','smoker','region']:
  encoded_column = pd.get_dummies(dataset[i], prefix=i + '_encod')
  encoded_columns.append(encoded_column)

# Concatenate all the encoded columns with the original dataset
encoded_dataset = pd.concat([dataset[['age', 'bmi', 'children']]] + encoded_columns, axis=1)

train = encoded_dataset

In [ ]:
labels = dataset['expenses']

In [ ]:
train_dataset, test_dataset, train_labels, test_labels = train_test_split(train,labels,test_size=0.2,random_state=20)

In [ ]:
x_train = train_dataset.astype(np.float32)
y_train = train_labels.astype(np.float32)

callbacks = EarlyStopping(monitor='mae', patience=35)
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(len(train.columns),)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae', 'mse'])

model.summary()

model.fit(x_train,y_train, epochs=200, batch_size=16, callbacks = [callbacks], verbose=1)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)


In [ ]:
print(test_predictions)